# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.grid_search import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
import pickle
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# load data from database
engine = create_engine('sqlite:///MessageCategoryInfo.db')
df = pd.read_sql("SELECT * FROM MessageCategoryInfo", engine)
X = df.message.values
Y = df.loc[:,'related':].values

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
 pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('moc', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
Y_pred = pipeline.predict(X_test)

In [8]:
category_names=df.loc[:,'related':].columns
for ind in range(36):
    accuracy=classification_report(Y_test[:,ind],Y_pred[:,ind])
    print ('Accuracy Result of Column '+category_names[ind])
    print(accuracy)
    

Accuracy Result of Column related
             precision    recall  f1-score   support

          0       0.63      0.36      0.45      1577
          1       0.81      0.93      0.87      4927
          2       0.80      0.08      0.15        50

avg / total       0.77      0.79      0.76      6554

Accuracy Result of Column request
             precision    recall  f1-score   support

          0       0.89      0.98      0.94      5481
          1       0.83      0.39      0.53      1073

avg / total       0.88      0.89      0.87      6554

Accuracy Result of Column offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6519
          1       0.00      0.00      0.00        35

avg / total       0.99      0.99      0.99      6554

Accuracy Result of Column aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3851
          1       0.75      0.51      0.60      2703

avg

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'moc', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'moc__estimator__bootstrap', 'moc__estimator__class_weight', 'moc__estimator__criterion', 'moc__estimator__max_depth', 'moc__estimator__max_features', 'moc__estimator__max_leaf_nodes', 'moc__estimator__min_impurity_decrease', 'moc__estimator__min_impurity_split', 'moc__estimator__min_samples_leaf', 'moc__estimator__min_samples_split', 'moc__estimator__min_weight_fraction_leaf', 'moc__estimator__n_estimators', 'moc__estimator__n_jobs', 'moc__estimator__oob_score', 'moc__estimator__random_state', 'moc__estimator__verbose', 'moc__estimator__

In [14]:
parameters = {'moc__estimator__n_estimators':[10,20],'moc__estimator__min_samples_split':[2,3]}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train,Y_train)

In [15]:
best_pipeline=cv.best_estimator_

In [16]:
Y_pred = best_pipeline.predict(X_test)

In [17]:
category_names=df.loc[:,'related':].columns
for ind in range(36):
    accuracy=classification_report(Y_test[:,ind],Y_pred[:,ind])
    print ('Accuracy Result of Column '+category_names[ind])
    print(accuracy)

Accuracy Result of Column related
             precision    recall  f1-score   support

          0       0.57      0.34      0.42      1533
          1       0.81      0.92      0.86      4976
          2       0.64      0.31      0.42        45

avg / total       0.76      0.78      0.76      6554

Accuracy Result of Column request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5491
          1       0.75      0.44      0.56      1063

avg / total       0.88      0.89      0.87      6554

Accuracy Result of Column offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.50      0.03      0.06        31

avg / total       0.99      1.00      0.99      6554

Accuracy Result of Column aid_related
             precision    recall  f1-score   support

          0       0.75      0.82      0.78      3920
          1       0.68      0.59      0.63      2634

avg

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [7]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [8]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('moc', MultiOutputClassifier(RandomForestClassifier()))
])


In [10]:
parameters = {'moc__estimator__n_estimators':[10,20],
              'moc__estimator__min_samples_split':[2,3]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [11]:
cv.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'moc__estimator__n_estimators': [10, 20], 'moc__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [18]:
best_pipeline=cv.best_estimator_

In [19]:
Y_pred = best_pipeline.predict(X_test)

In [20]:
category_names=df.loc[:,'related':].columns
for ind in range(36):
    accuracy=classification_report(Y_test[:,ind],Y_pred[:,ind])
    print ('Accuracy Result of Column '+category_names[ind])
    print(accuracy)

Accuracy Result of Column related
             precision    recall  f1-score   support

          0       0.62      0.35      0.44      1556
          1       0.82      0.93      0.87      4961
          2       0.57      0.32      0.41        37

avg / total       0.77      0.79      0.77      6554

Accuracy Result of Column request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5430
          1       0.83      0.38      0.52      1124

avg / total       0.87      0.88      0.86      6554

Accuracy Result of Column offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554

Accuracy Result of Column aid_related
             precision    recall  f1-score   support

          0       0.72      0.86      0.78      3822
          1       0.73      0.52      0.61      2732

avg

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [9]:
filename = 'finalized_model.sav'
model=pipeline
pickle.dump(model, open(filename, 'wb'))

In [10]:
# loaded_model = pickle.load(open(filename, 'rb'))
# loaded_model.predict(X_test)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.